In [1]:
import pickle
import pandas as pd
import random
import numpy as np
from collections import Counter
from sklearn.model_selection import GroupShuffleSplit

random.seed(123)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
PATH = "/home/parazit/ml_virus_host/v2.0/"
PATH_SAMPLE_IDS= PATH+"v3.0/sample_ids/"
PATH_DATA = PATH+"data/"

In [3]:
meta_df_genomes = pd.read_csv(PATH_DATA+"data_table.tsv", sep="\t", index_col = 0)

In [4]:
meta_df_genomes.head()

,virus tax id,virus name,virus family,host,host name,virus genus
refseq id,,,,,,
NC_001846,11138,Murine hepatitis virus,Coronaviridae,Mammalia,Mus musculus,Betacoronavirus
NC_026798,1587515,Black grass varicosavirus-like virus,Rhabdoviridae,Viridiplantae,Alopecurus myosuroides,Varicosavirus
NC_055213,2010276,Culex phasma-like virus,Phasmaviridae,Insecta,Culex quinquefasciatus,Orthophasmavirus
NC_001411,12285,Black beetle virus,Nodaviridae,Insecta,Heteronychus arator,Alphanodavirus
NC_016517,1127767,Espirito Santo virus,Birnaviridae,Insecta,Aedes albopictus,unclassified Birnaviridaevirus


In [5]:
def StratifiedGroupShuffleSplit(df_main, taxa_lvl):

    df_main = df_main.reindex(np.random.permutation(df_main.index)) # shuffle dataset

    # create empty train, val and test datasets
    df_train = pd.DataFrame()
    df_val = pd.DataFrame()
    df_test = pd.DataFrame()

    hparam_mse_wgt = 0.1 # must be between 0 and 1
    assert(0 <= hparam_mse_wgt <= 1)
    train_proportion = 0.6 # must be between 0 and 1
    assert(0 <= train_proportion <= 1)
    val_test_proportion = (1-train_proportion)/2

    subject_grouped_df_main = df_main.groupby([taxa_lvl], sort=False, as_index=False)
    category_grouped_df_main = df_main.groupby('host').count()[[taxa_lvl]]/len(df_main)*100

    def calc_mse_loss(df):
        grouped_df = df.groupby('host').count()[[taxa_lvl]]/len(df)*100
        df_temp = category_grouped_df_main.join(grouped_df, on = 'host', how = 'left', lsuffix = '_main')
        df_temp.fillna(0, inplace=True)
        df_temp['diff'] = (df_temp[taxa_lvl+'_main'] - df_temp[taxa_lvl])**2
        mse_loss = np.mean(df_temp['diff'])
        return mse_loss

    i = 0
    for _, group in subject_grouped_df_main:

        if (i < 3):
            if (i == 0):
                df_train = df_train.append(pd.DataFrame(group), ignore_index=True)
                i += 1
                continue
            elif (i == 1):
                df_val = df_val.append(pd.DataFrame(group), ignore_index=True)
                i += 1
                continue
            else:
                df_test = df_test.append(pd.DataFrame(group), ignore_index=True)
                i += 1
                continue

        mse_loss_diff_train = calc_mse_loss(df_train) - calc_mse_loss(df_train.append(pd.DataFrame(group), ignore_index=True))
        mse_loss_diff_val = calc_mse_loss(df_val) - calc_mse_loss(df_val.append(pd.DataFrame(group), ignore_index=True))
        mse_loss_diff_test = calc_mse_loss(df_test) - calc_mse_loss(df_test.append(pd.DataFrame(group), ignore_index=True))

        total_records = len(df_train) + len(df_val) + len(df_test)

        len_diff_train = (train_proportion - (len(df_train)/total_records))
        len_diff_val = (val_test_proportion - (len(df_val)/total_records))
        len_diff_test = (val_test_proportion - (len(df_test)/total_records)) 

        len_loss_diff_train = len_diff_train * abs(len_diff_train)
        len_loss_diff_val = len_diff_val * abs(len_diff_val)
        len_loss_diff_test = len_diff_test * abs(len_diff_test)

        loss_train = (hparam_mse_wgt * mse_loss_diff_train) + ((1-hparam_mse_wgt) * len_loss_diff_train)
        loss_val = (hparam_mse_wgt * mse_loss_diff_val) + ((1-hparam_mse_wgt) * len_loss_diff_val)
        loss_test = (hparam_mse_wgt * mse_loss_diff_test) + ((1-hparam_mse_wgt) * len_loss_diff_test)

        if (max(loss_train,loss_val,loss_test) == loss_train):
            df_train = df_train.append(pd.DataFrame(group), ignore_index=True)
        elif (max(loss_train,loss_val,loss_test) == loss_val):
            df_val = df_val.append(pd.DataFrame(group), ignore_index=True)
        else:
            df_test = df_test.append(pd.DataFrame(group), ignore_index=True)

        #print ("Group " + str(i) + ". loss_train: " + str(loss_train) + " | " + "loss_val: " + str(loss_val) + " | " + "loss_test: " + str(loss_test) + " | ")
        i += 1

    return df_train, df_val, df_test

Выбираем целевуе переменную (хозяин) и столбец с нужным таксономическим уровнем вирусов (род)

In [6]:
df = meta_df_genomes[["host", "virus genus"]].reset_index()

Разбиваем данные на 3 подвыборки (df_train, df_val, df_test) до тех пор, пока каждая из выборок будет содержать не менее 200 геномов

In [7]:
df_train, df_val, df_test = StratifiedGroupShuffleSplit(df, "virus genus")
while len(df_train) < 200 or len(df_val) < 200 or len(df_test) < 200:
    df_train, df_val, df_test = StratifiedGroupShuffleSplit(df, "virus genus")

Функция для вывода словаря с названиями хозяев на русском

In [8]:
def rus_dict(df_inside):
    out = dict(zip(["Насекомые", "Млекопитающие", "Растения"], \
             dict(sorted({k:round((v*100/len(df_inside)), 2) for k, v in Counter(df_inside.host).items()}.items())).values()))
    print(str(out)[1:-1])
    return

Печатаем статистику разбиения и решаем, подходит ли оно

In [9]:
print("Подвыборка №1")    
le = len(df_train)
print("Размер: "+ str(le) + " геномов, " + str(round(le*100/1390, 2)) + "% от исходного датасета")
rus_dict(df_train)
print("\n")

print("Подвыборка №2")  
le = len(df_val)
print("Размер: "+ str(le) + " геномов, " + str(round(le*100/1390, 2)) + "% от исходного датасета")
rus_dict(df_val)
print("\n")

print("Подвыборка №3")
le = len(df_test)
print("Размер: "+ str(le) + " геномов, " + str(round(le*100/1390, 2)) + "% от исходного датасета")
rus_dict(df_test)
print("\n")

Подвыборка №1
Размер: 217 геномов, 15.61% от исходного датасета
'Насекомые': 33.64, 'Млекопитающие': 44.7, 'Растения': 21.66


Подвыборка №2
Размер: 900 геномов, 64.75% от исходного датасета
'Насекомые': 23.44, 'Млекопитающие': 49.33, 'Растения': 27.22


Подвыборка №3
Размер: 273 геномов, 19.64% от исходного датасета
'Насекомые': 28.94, 'Млекопитающие': 49.08, 'Растения': 21.98




Если разбиение подходит, сохраняем в последовательности "тренировочная, валидационная, тестовая" выборки

In [10]:
train_ids_g = df_test["refseq id"].values
val_ids_g = df_train["refseq id"].values
test_ids_g = df_val["refseq id"].values

output_samples = [train_ids_g, val_ids_g, test_ids_g]

In [ ]:
with open("/home/parazit/ml_virus_host/v2.0/v3.0/sample_ids/FILENAME.pkl", 'wb') as file:
    pickle.dump(output_samples, file)

# Состав выборок с непересекающимися родами

5 итераций, разбиение - непересекающиеся рода. Итерация №0 использовалась для подбора параметров моделей, для оставшихся итераций использовались те же параметры.

In [11]:
samples_lst = []
for n in range(5):
    samples_lst.append(pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_genomes_genera_it"+str(n)+".pkl", "rb")))
    
    
i=-1

for iteration in samples_lst:
    i+=1
    print("Разбиение №"+str(i))
    print("Train : Validation : Test")
    print(str(len(iteration[0])) + " : " + str(len(iteration[1])) + " : " + str(len(iteration[2])))
    print(str(round(len(iteration[0])*100/1390, 2)) +"%" + " : " + str(round(len(iteration[1])*100/1390, 2))+"%" + " : " + str(round(len(iteration[2])*100/1390, 2))+"%")
    print("\n")

Разбиение №0
Train : Validation : Test
829 : 223 : 338
59.64% : 16.04% : 24.32%


Разбиение №1
Train : Validation : Test
627 : 459 : 304
45.11% : 33.02% : 21.87%


Разбиение №2
Train : Validation : Test
766 : 281 : 343
55.11% : 20.22% : 24.68%


Разбиение №3
Train : Validation : Test
854 : 253 : 283
61.44% : 18.2% : 20.36%


Разбиение №4
Train : Validation : Test
902 : 235 : 253
64.89% : 16.91% : 18.2%




# Создание выборок фрагментов (случайно отбираем по 2 фрагмента на геном)

In [12]:
meta_df_800 = pd.read_csv(PATH_DATA+"data_table_800.tsv", sep="\t", index_col = 3)
meta_df_400 = pd.read_csv(PATH_DATA+"data_table_400.tsv", sep="\t", index_col = 3)

Столбец "gbac" содержит id фрагментов 

In [13]:
meta_df_800.head()

,gbac,host,family,genus
genomes_ids,,,,
NC_001846,NC_001846_0,Mammalia,Coronaviridae,Betacoronavirus
NC_001846,NC_001846_1,Mammalia,Coronaviridae,Betacoronavirus
NC_001846,NC_001846_2,Mammalia,Coronaviridae,Betacoronavirus
NC_001846,NC_001846_3,Mammalia,Coronaviridae,Betacoronavirus
NC_001846,NC_001846_4,Mammalia,Coronaviridae,Betacoronavirus


Разбиение геномов на обучающую, валидационную и тестовую выборки

In [14]:
sample_ids_genomes = pickle.load(open(PATH_SAMPLE_IDS + "train_val_test_genomes_genera_it0.pkl", "rb"))

Функция для выделения id фрагментов, соотвествующих геномам

In [15]:
def extract_fragments(meta_df_fragments, sample_ids_genomes, length, taxa_level):
    
    # отбираем по 2 фрагмета на геном
    train_ids = meta_df_fragments.loc[sample_ids_genomes[0]].groupby("genomes_ids").sample(2).gbac.values
    val_ids = meta_df_fragments.loc[sample_ids_genomes[1]].groupby("genomes_ids").sample(2).gbac.values
    test_ids = meta_df_fragments.loc[sample_ids_genomes[2]].groupby("genomes_ids").sample(2).gbac.values
    
    # сохраняем в последовательности "обучающая, валидационная, тестовая" выборки
    output = [train_ids, val_ids, test_ids]

    with open("/home/parazit/ml_virus_host/v2.0/v3.0/sample_ids/train_val_test_+"+str(length)+"+_"+taxa_level+".pkl", 'wb') as file:
        pickle.dump(output, file)

Пример использования

In [ ]:
extract_fragments(meta_df_800, sample_ids_genomes, "800", "genera")